In [1]:
%pylab inline
from rpyc import connect
#from PyRedPitaya.pc import RedPitaya
from PyRedPitaya.client_memory import ClientMemory
from PyRedPitaya.memory import MemoryInterface
from PyRedPitaya.enum import Enum
from PyRedPitaya.instrument import *
import time
REDPITAYA_IP = '192.168.14.241'

Populating the interactive namespace from numpy and matplotlib


In [2]:
class Scope2(MemoryInterface):
    data_length = 2**12
    def __init__(self, addr_base = 0x40100000, channel='A', **kwd):
        kwd["addr_base"] = addr_base
        super(Scope2, self).__init__(**kwd)

    writestate_machine_bit = GetSetBit(addr=0x0, pos=1)
    trigger_bit =  GetSetBit(addr=0x0, pos=0)
    
    def reset_writestate_machine(self,v=True):
        self.writestate_machine_bit = v
    
    def arm_trigger(self,v=True):
        self.trigger_bit = v
    
    trigger_source = GetSetRegister(0x4, TriggerSource())
    threshold_ch1 = GetSetRegister(0x8, SignedInteger(size=14))
    threshold_ch2 = GetSetRegister(0xC, SignedInteger(size=14))
    trigger_delay = GetSetRegister(0x10, UnsignedInteger(size=32))
    data_decimation = GetSetRegister(0x14, Decimation())
    write_pointer_current = GetRegister(0x18, UnsignedInteger(size=14))
    write_pointer_trigger = GetRegister(0x1C, UnsignedInteger(size=14))
    hysteresis_ch1 = GetSetRegister(0x20, SignedInteger(size=14))
    hysteresis_ch2 = GetSetRegister(0x24, SignedInteger(size=14))
    average = GetSetBit(addr=0x28, pos=0)
    #equalization filter not implemented here
    dac2_on_ch1 = GetSetBit(0x50, pos=0)
    dac1_on_ch2 = GetSetBit(0x50, pos=1)
    
    hw_avgs = GetSetRegister(0xAC, UnsignedInteger(size=18))
    version = GetRegister(0xB0, UnsignedInteger(size=32))
    hw_avg_status = GetRegister(0xB4, UnsignedInteger(size=32))
    adc_trigged = GetSetBit(0xB4, pos=3)
    npt_mode = GetSetBit(0xB4, pos=2)
    avg_mode = GetSetBit(0xB4, pos=1)        
    avg_do = GetSetBit(0xB4, pos=0)
    avg_cnt= GetRegister(0xB8, UnsignedInteger(size=18))

    #### Function specific to read the array of data
    def get_rawdata(self, addr):
        x = self.reads(addr, self.data_length)
        y = x.copy()
        y.dtype = np.int32
        y[y>2**31] -= 2**31
        return y
        
    @property
    def rawdata_ch1(self):
        return self.get_rawdata(0x10000)
    @property
    def rawdata_ch2(self):
        return self.get_rawdata(0x20000)
    
    @property
    def data_ch1(self):
        return np.roll(self.rawdata_ch1,-int(self.write_pointer_trigger))
    @property
    def data_ch2(self):
        return np.roll(self.rawdata_ch2,-int(self.write_pointer_trigger))
    

    # helpers
    @property
    def times(self):
        return np.linspace(0.0,8e-9*self.data_decimation*float(self.data_length),self.data_length,endpoint=False)
    
    def setup(self,frequency=1,trigger_source=TriggerSource.immediately):
        self.reset_writestate_machine(v=True)
        self.trigger_delay = self.data_length
        self.dac1_on_ch2 = False
        self.dac2_on_ch1 = False
        self.arm_trigger(v=False)
        self.average = True
        self.frequency = frequency
        self.trigger_source = trigger_source
        self.reset_writestate_machine(v=False)
        self.arm_trigger()
    
    def rearm(self,frequency=None,trigger_source = 8):
        if not frequency is None:
            self.frequency = frequency
        self.trigger_delay = self.data_length
        self.trigger_source = trigger_source
        self.arm_trigger()
    
    @property
    def frequency(self):
        return 1.0/float(self.data_decimation)/float(self.data_length)/8e-9
    
    @frequency.setter
    def frequency(self, v):
        fbase = 125e6/float(2**14)
        factors = [1,8,64,1024,8192,65536,65537]
        for f in factors:
            if v > fbase/float(f):
                self.data_decimation = f
                break
            if f == 65537:
                self.data_decimation = 65536
                print "Frequency too low: Impossible to sample the entire waveform"



class RedPitaya(ClientMemory):
    hk = InterfaceDescriptor(HK)
    ams = InterfaceDescriptor(AMS)
    scope = InterfaceDescriptor(Scope2)
    pid11 = InterfaceDescriptor(Pid,number='11')
    pid12 = InterfaceDescriptor(Pid,number='12')
    pid21 = InterfaceDescriptor(Pid,number='21')
    pid22 = InterfaceDescriptor(Pid,number='22')
    asga = InterfaceDescriptor(ASG,channel='A')
    asgb = InterfaceDescriptor(ASG,channel='B')


In [3]:
conn = connect(REDPITAYA_IP, port=18861)
redpitaya = RedPitaya(conn)
rp=redpitaya

print "FPGA code version: %d" % rp.scope.version

timeout: timed out

In [ ]:
print ".bit version: %d" % rp.scope.version

rp.scope.reset_writestate_machine(v=True)
rp.scope.trigger_delay = rp.scope.data_length
rp.scope.dac1_on_ch2 = False
rp.scope.dac2_on_ch1 = False
rp.scope.data_decimation = 1
rp.scope.arm_trigger(v=False)
rp.scope.average = False
rp.scope.trigger_source = 4
rp.scope.threshold_ch1 = 4000
rp.scope.threshold_ch2 = 4000
rp.scope.hw_avgs=1000
rp.scope.reset_writestate_machine(v=False)
rp.scope.arm_trigger()

#for ii in range(avgs):
#    rp.scope.trigger_source=4
while (rp.scope.avg_cnt<rp.scope.hw_avgs):
    time.sleep(10E-3)

subplot(111,xlabel="Time (us)", ylabel="Voltage", title="Scope")
plot (rp.scope.times*1e6,rp.scope.rawdata_ch1/rp.scope.avg_cnt,'r')
plot (rp.scope.times*1e6,rp.scope.rawdata_ch2/rp.scope.avg_cnt,'b')
#xlim(0,1)
print "hw_avgs: %d, avg_cnt: %d, adc_trigged: %d, npt_mode: %d, avg_mode: %d, avg_do: %d" %(rp.scope.hw_avgs,rp.scope.avg_cnt,rp.scope.adc_trigged, rp.scope.npt_mode, rp.scope.avg_mode, rp.scope.avg_do )

In [ ]:
def simulated_pulse(state, kappa, gamma, eta, tpts):
    tup=random.exponential(1./(2.*pi*gamma))
    #tup=1./(2.*pi*gamma)*random()
    return (1-exp(-kappa*tpts))*exp(-kappa*(tpts>tup)*(tpts-tup))*state+random.normal(0,eta,len(tpts))

In [ ]:
tpts=linspace(0,10e-6,101)
ans=zeros(len(tpts))
for n in arange(1,1000):
    ans=(ans*(n-1)+simulated_pulse(state=1,kappa=1e6,gamma=1e5,eta=1,tpts=tpts))/n
plot(tpts,simulated_pulse(state=1,kappa=1e6,gamma=1e5,eta=.25,tpts=tpts))
plot(tpts,ans)